# Cross-Encoder(evaluates the pair (query + chunk) using a BERT-style encoder with attention)

## Notebook Summary: Hybrid QA with Cross-Encoder Reranking + Compound + Procedural QA

This notebook implements a robust RAG-based extractive QA system for telecom documents, integrating multiple enhancements to improve factual accuracy and interpretability.

### Key Features:

1. **Cross-Encoder Reranking**  
   Improves retrieval quality by reranking initial FAISS results using `cross-encoder/ms-marco-MiniLM-L-6-v2`, allowing deeper semantic alignment between query and chunks.

2. **Compound Question Decomposition**  
   Automatically splits multi-clause questions (e.g., with "and", "or") and answers each clause individually using separate prompts.

3. **Procedural Multi-Span Extraction**  
   For procedural or stepwise queries, uses regex-based patterns to extract actionable steps directly from context.

4. **Auto-Routing Strategy**  
   The pipeline selects between standard extractive QA and procedural span extraction based on query structure.

5. **Evaluation Metrics**  
   Evaluated on 100 curated QA pairs using:
   - **SQuAD (EM / F1)**
   - **ROUGE-L**
   - **BLEU**

This setup demonstrates the most advanced variant of the centralized RAG system in the project, optimized for telecom-specific factual and procedural queries.

In [1]:
from pathlib import Path
import faiss
import pickle
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer

In [2]:
# Load FAISS index and chunks
index_path = "/mnt/data/RAG/3gpp_index.faiss"
chunks_path = "/mnt/data/RAG/3gpp_chunks.pkl"

index = faiss.read_index(index_path)
with open(chunks_path, "rb") as f:
    documents = pickle.load(f)

# Load embedding model used for indexing
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
def retrieve_context(query, top_k=3):
    query_emb = embedding_model.encode([query], normalize_embeddings=True)
    D, I = index.search(query_emb.astype("float32"), top_k)
    return [documents[i] for i in I[0]]

def retrieve_with_rerank(query, top_k=5):
    # Step 1 — initial FAISS search
    query_emb = embedding_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(query_emb).astype("float32"), top_k * 2)  # wider net

    initial_results = [documents[i] for i in I[0]]

    # Step 2 — prepare (query, chunk) pairs
    pairs = [(query, doc["content"]) for doc in initial_results]

    # Step 3 — rerank with cross-encoder
    scores = reranker.predict(pairs)
    reranked = sorted(zip(scores, initial_results), key=lambda x: x[0], reverse=True)[:top_k]

    return [doc for _, doc in reranked]

In [3]:
from sentence_transformers import CrossEncoder

# Load Cross-Encoder model once
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [4]:
SYSTEM_PROMPT = (
    "You are a precise assistant. Extract the exact answer span from the context. "
    "Do not paraphrase, summarize, or add extra information. "
    "The answer must appear exactly in the context."
    "If the context lists multiple conditions, actions, or branches, include them all as written. "
    "Do not summarize or paraphrase — copy the exact text from the context, line by line."
)

def build_rag_prompt(context_chunks, question):
    combined_context = "\n\n".join([chunk['content'] for chunk in context_chunks])
    user_prompt = (
        f"Context: {combined_context}\n\n"
        f"Question: {question}\n"
        f"Answer from the context only:"
    )
    return f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n{user_prompt} [/INST]"

model_path = "/mnt/data/llama2_qa_lora_output5/final"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).to("cuda")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
def clean_prediction(raw_text):
    # Remove everything before the last [INST]
    answer = raw_text.split("[/INST]")[-1].strip()

    # Remove strange characters
    answer = re.sub(r"[^\w\s\-.,:/()]", "", answer)

    # Remove repeating phrases like "The key is... The key is... The key is..."
    answer = re.sub(r'(\b.+?:)(\s*\1)+', r'\1', answer)

    # Trim repetitive word loops (e.g., "structured as follows" x 5)
    tokens = answer.split()
    for i in range(1, len(tokens) // 2):
        if tokens[:i] == tokens[i:2*i]:
            answer = " ".join(tokens[:i])
            break

    # Optionally truncate to sentence boundary
    sentence_end = re.search(r'[.?!]', answer)
    if sentence_end:
        answer = answer[:sentence_end.end()]

    return answer.strip()

In [6]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk

STOPWORDS = set(stopwords.words("english"))

import re

def normalize(text):
    return re.sub(r'\W+', ' ', text.lower())

def lexical_overlap(query, chunk):
    q_tokens = set(normalize(query).split()) - STOPWORDS
    c_tokens = set(normalize(chunk).split()) - STOPWORDS
    return len(q_tokens & c_tokens) / (len(q_tokens | c_tokens) + 1e-5)

def tfidf_score(query, chunk, vectorizer=None):
    docs = [query, chunk]
    if not vectorizer:
        vectorizer = TfidfVectorizer().fit(docs)
    vecs = vectorizer.transform(docs)
    return (vecs[0] @ vecs[1].T).A[0][0]

In [7]:
def split_compound_question(q):
    parts = re.split(r"\band\b|\bor\b|[,;]", q)
    return [p.strip() for p in parts if len(p.strip().split()) > 3]


def answer_with_rag_llama(question, top_k=5, verbose=False):
    retrieved = retrieve_with_rerank(question, top_k=top_k)

    sub_qs = split_compound_question(question)

    # Handle compound question (multi-prompt)
    if len(sub_qs) > 1:
        answers = []
        for sq in sub_qs:
            sub_prompt = build_rag_prompt(retrieved, sq)
            raw = qa_pipeline(
                sub_prompt, 
                max_new_tokens=160, 
                do_sample=False, 
                eos_token_id=tokenizer.eos_token_id, 
                pad_token_id=tokenizer.eos_token_id
            )[0]["generated_text"]

            ans = clean_prediction(raw)
            answers.append(f"→ {sq}: {ans}")

        full_answer = "\n".join(answers)

        # Context containment check (on full answer)
        all_context = " ".join([c["content"] for c in retrieved])
        if not any(ans.split(": ", 1)[-1] in all_context for ans in answers):
            print("🚨 One or more sub-answers not found in context — check retrieval or generation.")
        return full_answer, retrieved

    # Handle simple (single-clause) question
    prompt = build_rag_prompt(retrieved, question)
    raw_output = qa_pipeline(
        prompt, 
        max_new_tokens=160, 
        do_sample=False, 
        eos_token_id=tokenizer.eos_token_id, 
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    answer = clean_prediction(raw_output)

    # Sanity check
    if len(answer.split()) < 2 or len(answer.split()) > 40:
        print("⚠️ Warning: Possibly bad output. Check content or retrieval.")

    # ✅ Context containment validation
    all_context = " ".join([c["content"] for c in retrieved])
    if answer not in all_context:
        print("🚨 Answer not found in retrieved context — check prompt or retrieval quality.")

    if verbose:
        print("📌 Prompt:\n", prompt)
        print("\n🧾 Raw Output:\n", raw_output)
        print("\n✅ Cleaned Answer:", answer)
        for i, chunk in enumerate(retrieved):
            print(f"\n--- Context {i+1} ---")
            print(chunk["content"])

    return answer, retrieved

import re

def extract_multi_spans(context: str) -> list:
    """Extract procedural-style sentences from telecom context using regex patterns."""
    spans = []

    # Patterns that catch conditional and action rules
    patterns = [
        r"(?i)(?:upon|when|if|after|before).+?shall.+?[.;]",  # conditional + shall
        r"(?i)the (?:ue|amf|network|nas|gnb).+?shall.+?[.;]",  # direct instructions
        r"(?i)-\s*.+?shall.+?[.;]",  # bullet points with 'shall'
        r"(?i)the (?:ue|amf|nas|network).+?enters.+?[.;]",     # entry triggers
    ]

    for pattern in patterns:
        matches = re.findall(pattern, context)
        spans.extend(matches)

    import difflib

    def is_similar(a, b, threshold=0.85):
        return difflib.SequenceMatcher(None, a, b).ratio() > threshold
    
    unique_spans = []
    for s in spans:
        cleaned = s.strip()
        if not any(is_similar(cleaned, u) for u in unique_spans):
            unique_spans.append(cleaned)
    
    return unique_spans[:6]

def answer_with_rag_llama_multispan(question, top_k=5, verbose=False):
    retrieved = retrieve_with_rerank(question, top_k=top_k)
    combined_context = " ".join([re.sub(r'\s+', ' ', c["content"]) for c in retrieved])
    spans = extract_multi_spans(combined_context)  # uses regex

    if not spans:
        return "⚠️ No clear steps found in context.", retrieved

    final = "\n".join([f"• {s.strip()}" for s in spans])
    return final, retrieved

def route_question_to_best_strategy(question, top_k=5, verbose=False):
    # Use multi-span for procedural questions
    if any(q in question.lower() for q in ["steps", "procedures", "when does", "what happens if", "if the ue"]):
        return answer_with_rag_llama_multispan(question, top_k=top_k, verbose=verbose)
    else:
        return answer_with_rag_llama(question, top_k=top_k, verbose=verbose)

In [8]:
import json
from tqdm import tqdm
from evaluate import load

# Load QA pairs
def load_qa_pairs(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

qa_pairs = load_qa_pairs("3gpp_qa_100_pairs.jsonl")

# Load metrics
squad_metric = load("squad")
rouge = load("rouge")
bleu = load("bleu")

bleu_predictions = []
bleu_references = []
results = []

for sample in tqdm(qa_pairs):
    question = sample["question"]
    reference = sample["answer"]

    try:
        prediction, _ = route_question_to_best_strategy(question)
    except Exception as e:
        print(f"⚠️ Error on: {question}\n{e}")
        prediction = ""

    # Add to metrics
    squad_metric.add(
        prediction={"id": str(hash(question)), "prediction_text": prediction},
        reference={"id": str(hash(question)), "answers": {"text": [reference], "answer_start": [0]}}
    )
    rouge.add(prediction=prediction, reference=reference)
    bleu_predictions.append(prediction)
    bleu_references.append([reference])
    results.append({
        "question": question,
        "reference": reference,
        "prediction": prediction
    })

# Compute final scores
squad_scores = squad_metric.compute()
rouge_scores = rouge.compute()
bleu_score = bleu.compute(predictions=bleu_predictions, references=bleu_references)["bleu"]

# Print results
print("\n📊 Final Evaluation Results (Setup 4 — Cross-Encoder + Compound + Procedural):")
print(f"Exact Match (EM): {squad_scores['exact_match']:.2f}")
print(f"F1 Score        : {squad_scores['f1']:.2f}")
print(f"ROUGE-L         : {rouge_scores['rougeL']:.4f}")
print(f"BLEU            : {bleu_score:.4f}")

  1%|▍                                          | 1/100 [00:03<06:09,  3.73s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


⚠️ Warning: Possibly bad output. Check content or retrieval.
🚨 Answer not found in retrieved context — check prompt or retrieval quality.


  2%|▊                                          | 2/100 [00:05<03:45,  2.30s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 Answer not found in retrieved context — check prompt or retrieval quality.


  3%|█▎                                         | 3/100 [00:12<07:51,  4.86s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

 98%|█████████████████████████████████████████▏| 98/100 [08:29<00:06,  3.35s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚨 Answer not found in retrieved context — check prompt or retrieval quality.


100%|█████████████████████████████████████████| 100/100 [08:33<00:00,  5.14s/it]

🚨 Answer not found in retrieved context — check prompt or retrieval quality.



📊 Final Evaluation Results (Setup 4 — Cross-Encoder + Compound + Procedural):
Exact Match (EM): 0.00
F1 Score        : 19.51
ROUGE-L         : 0.2018
BLEU            : 0.0092
